In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import (
    denumpyize,
    numpyize,
    candidate_to_live_config,
    calc_spans,
    analyze_fills,
    create_xk,
    round_dynamic,
    round_values,
    sort_dict_keys,
)
from procedures import (
    dump_live_config,
    load_live_config,
    add_argparse_args,
    prepare_backtest_config,
    make_get_filepath,
)
from njit_funcs_recursive_grid import *
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]
plt.rcParams["figure.facecolor"] = "w"
pd.set_option("precision", 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = "configs/backtest/default.hjson"
        self.exchange = "binance"
        self.symbol = "XMRUSDT"
        self.market_type = "futures"
        self.user = "binance_01"
        self.start_date = "2021-05-01"
        self.end_date = "2022-01-21"
        self.starting_balance = 10000.0
        self.starting_configs = ""
        self.base_dir = "backtests"


config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f"millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms")

In [ ]:
df = pd.DataFrame({"timestamp": timestamps, "qty": qtys, "price": prices}).set_index("timestamp")
df.price.iloc[::100].plot(title="Coin Price", xlabel="Time", ylabel="Price")

In [ ]:
# choose a slice on which to test
wsize_days = 365
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "long": {
        "enabled": True,
        "ema_span_0": 1036.4758617491368,
        "ema_span_1": 1125.5167077975314,
        "iqty_pct": 0.01,
        "iprice_ema_dist": -0.02,
        "wallet_exposure_limit": 1.0,
        "ddown_factor": 0.6,
        "rentry_pprice_dist": 0.015,
        "rentry_pprice_dist_wallet_exposure_weighting": 15,
        "min_markup": 0.02,
        "markup_range": 0.02,
        "n_close_orders": 7,
        "auto_unstuck_wallet_exposure_threshold": 0.15,
        "auto_unstuck_ema_dist": 0.02,
    }
}
hand_tuned["short"] = hand_tuned["long"].copy()
hand_tuned["short"]["enabled"] = False
config_to_test = {**config, **hand_tuned}

In [ ]:
balance = config_to_test["starting_balance"]
highest_bid = df.price.iloc[-1]

inverse = config_to_test["inverse"]
do_long = True
qty_step = config_to_test["qty_step"]
price_step = config_to_test["price_step"]
min_qty = config_to_test["min_qty"]
min_cost = config_to_test["min_cost"]
c_mult = config_to_test["c_mult"]

iqty_pct = config_to_test["long"]["iqty_pct"]
iprice_ema_dist = config_to_test["long"]["iprice_ema_dist"]
ddown_factor = config_to_test["long"]["ddown_factor"]
rentry_pprice_dist = config_to_test["long"]["rentry_pprice_dist"]
rentry_pprice_dist_wallet_exposure_weighting = config_to_test["long"][
    "rentry_pprice_dist_wallet_exposure_weighting"
]
wallet_exposure_limit = config_to_test["long"]["wallet_exposure_limit"]
auto_unstuck_ema_dist = config_to_test["long"]["auto_unstuck_ema_dist"]
auto_unstuck_wallet_exposure_threshold = config_to_test["long"]["auto_unstuck_wallet_exposure_threshold"]

psize = 0.0
pprice = 0.0
ema_band_lower = highest_bid

grid = calc_long_entries(
    balance,
    psize,
    pprice,
    highest_bid,
    ema_band_lower,
    inverse,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    c_mult,
    iqty_pct,
    iprice_ema_dist,
    ddown_factor,
    rentry_pprice_dist,
    rentry_pprice_dist_wallet_exposure_weighting,
    wallet_exposure_limit,
    auto_unstuck_ema_dist,
    auto_unstuck_wallet_exposure_threshold,
)

gdf = pd.DataFrame(grid, columns=["qty", "price", "type", "psize", "pprice", "wallet_exposure"])
gdf.loc[:, "eprice_pprice_diff"] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:, "ddown_factor"] = gdf.qty / gdf.psize.shift()
gdf.loc[:, "bkr_price"] = gdf.apply(
    lambda x: calc_bankruptcy_price(balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult),
    axis=1,
)
colors = "rbygcmk"

# Display grid on graph
# -- = pprice = new position price
# - = price = DCA entry
n_ticks = 60 * 60 * 24 * 14
timedt = pd.to_datetime(timestamps[-n_ticks:], unit="ms")
dfx = pd.DataFrame(
    {"timestamp": timedt, "qty": qtys[-n_ticks:], "price": prices[-n_ticks:]}
).set_index("timestamp")
lastdayfrom = pd.to_datetime(Args().end_date)
# lastdayfrom = pd.Timestamp('2021-05-21')
fig = (
    dfx.price.loc[lastdayfrom - pd.Timedelta(days=280) : lastdayfrom]
    .iloc[::100]
    .plot(title="Grid position", xlabel="Time", ylabel="Price")
)
for i, e in enumerate(gdf.itertuples()):
    fig.axhline(y=e.price, color=f"{colors[i%len(colors)]}", linestyle="-")
    # plt.axhline(y=e.pprice, color=f"{colors[i%len(colors)]}", linestyle="--")
a = (1 - gdf.iloc[[0, -1]]["price"].iat[-1] / gdf.iloc[[0, -1]]["price"].iat[0]) * 100
print("\nLong Grid Span = {:.2f} %\n".format(round(a, 2)))
gdf

In [ ]:
sts = time()
fills, stats = backtest(config_to_test, dataslice, recursive_grid=True)
elapsed = time() - sts
print(f"seconds elapsed {elapsed:.4f}")
fdf, sdf, analysis = analyze_fills(fills, stats, config_to_test)
pprint.pprint(analysis)
fdf

In [ ]:
sdf

In [ ]:
sdf.balance.plot()
sdf.equity.plot(title="Balance and equity", xlabel="Time", ylabel="Balance")

In [ ]:
plot_fills(df, fdf, plot_whole_df=True, title="Overview Fills")

In [ ]:
fdf[fdf.psize > 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Long Position size",
)
fdf[fdf.psize < 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Short Position size",
)

In [ ]:
sdf.price.plot(title="Average entry price", xlabel="Time", ylabel="Price")
sdf.long_pprice.replace(0.0, np.nan).plot()
sdf.short_pprice.replace(0.0, np.nan).plot()

In [ ]:
lpprices = sdf[sdf.long_pprice != 0.0]
padistance_long = (lpprices.long_pprice - lpprices.price).abs() / lpprices.price
print(f"Mean price action distance long {padistance_long.mean():.6f}")
padistance_long.plot(title="Price action distance", xlabel="Time", ylabel="Price action distance")

In [ ]:
spprices = sdf[sdf.short_pprice != 0.0]
padistance_short = (spprices.short_pprice - spprices.price).abs() / spprices.price
print(f"Mean price action distance short {padistance_short.mean():.6f}")
padistance_short.plot(title="Price action distance", xlabel="Time", ylabel="Price action distance")

In [ ]:
# Inspect long EMAs
# blue: lower unstucking; red: upper unstucking; green: initial entry
spans = [
    config_to_test["long"]["ema_span_0"],
    (config_to_test["long"]["ema_span_0"] * config_to_test["long"]["ema_span_1"]) ** 0.5,
    config_to_test["long"]["ema_span_1"],
]
print(
    f"spans in minutes {spans}",
    f"n_days {(df.index[-1] - df.index[0]) / 1000 / 60 / 60 / 24:.1f}",
)
for i in range(3):
    # change to seconds
    spans[i] *= 60
emas = pd.DataFrame({str(span): df.price.ewm(span=span, adjust=False).mean() for span in spans})
ema_band_lower = emas.min(axis=1)
unstucking_band_lower = ema_band_lower * (1 - config_to_test["long"]["auto_unstuck_ema_dist"])
ema_band_upper = emas.max(axis=1)
unstucking_band_upper = ema_band_upper * (1 + config_to_test["long"]["auto_unstuck_ema_dist"])
long_ientry_band = ema_band_lower * (1 - config_to_test["long"]["iprice_ema_dist"])
df.iloc[::100].price.plot(style="y-", title="Unstucking Bands and Initial Entry Band")
unstucking_band_lower.iloc[::100].plot(
    style="b--",
)
unstucking_band_upper.iloc[::100].plot(style="r--")
long_ientry_band.iloc[::100].plot(style="g-.")

In [ ]:
# Inspect short EMAs
# blue: lower unstucking; red: upper unstucking; green: initial entry
spans = [
    config_to_test["short"]["ema_span_0"],
    (config_to_test["short"]["ema_span_0"] * config_to_test["short"]["ema_span_1"]) ** 0.5,
    config_to_test["short"]["ema_span_1"],
]
print(
    f"spans in minutes {spans}",
    f"n_days {(df.index[-1] - df.index[0]) / 1000 / 60 / 60 / 24:.1f}",
)
for i in range(3):
    # change to seconds
    spans[i] *= 60
emas = pd.DataFrame({str(span): df.price.ewm(span=span, adjust=False).mean() for span in spans})
ema_band_lower = emas.min(axis=1)
unstucking_band_lower = ema_band_lower * (1 - config_to_test["short"]["auto_unstuck_ema_dist"])
ema_band_upper = emas.max(axis=1)
unstucking_band_upper = ema_band_upper * (1 + config_to_test["short"]["auto_unstuck_ema_dist"])
short_ientry_band = ema_band_upper * (1 + config_to_test["short"]["iprice_ema_dist"])
df.iloc[::100].price.plot(style="y-", title="Unstucking Bands and Initial Entry Band")
unstucking_band_lower.iloc[::100].plot(style="b--")
unstucking_band_upper.iloc[::100].plot(style="r--")
short_ientry_band.iloc[::100].plot(style="g-.")